In [ ]:
import backtrader as bt
import datetime

import pandas as pd

In [ ]:
# 保留 pandas feed 部分

# 数据准备 300ETF 日级别数据
dataframe = pd.DataFrame()
for i in range(7):
    df = pd.read_csv('../hist_data/510300_D_{}.csv'.format(2013+i), parse_dates=True, index_col=0)
    dataframe = pd.concat([dataframe,df])
    
# pandasdata feeder
feed = bt.feeds.PandasData(dataname=dataframe, openinterest=None)

In [ ]:
class MacdStrategy(bt.Strategy):
    params = (
        ('fastperiod', 10),
        ('slowperiod', 22),
        ('signalperiod', 8),
    )
    
        
    def __init__(self):
        
        kwargs = {
            'fastperiod': self.p.fastperiod,
            'fastmatype': bt.talib.MA_Type.EMA,
            'slowperiod': self.p.slowperiod,
            'slowmatype': bt.talib.MA_Type.EMA,
            'signalperiod': self.p.signalperiod,
            'signalmatype': bt.talib.MA_Type.EMA,
        }

        # Add a Macd indicator
        self.macd = bt.talib.MACDEXT(
             self.data0.close, **kwargs)
    
        self.crossover = bt.indicators.CrossOver(self.macd.macd, self.macd.macdsignal, plot=False)
        
        self.buy_signal = (self.crossover==1)
        self.sell_signal = (self.crossover==-1)
       
        self.pa = bt.talib.CDLXSIDEGAP3METHODS(
        self.data0.open,self.data0.high,self.data0.low,self.data0.close)
        # To keep track of pending orders
        self.order = None
        
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        if order.status in [order.Completed, order.Canceled, order.Margin, order.Rejected]:
            # Write down: no pending order
            self.order = None

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # Check if we are in the market
        if not self.position:
            # Not yet ... we MIGHT BUY if ...
            if self.buy_signal[0] and self.macd.macd[0]>0.0 and self.macd.macdsignal[0]>0.0:
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
        else:
            # Already in the market ... we might sell
            if self.sell_signal[0]:
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
cerebro = bt.Cerebro()


cerebro.adddata(feed, name= 'etf300')
cerebro.addstrategy(MacdStrategy)


# 小场面1万起始资金
cerebro.broker.setcash(10000.0)

# 手续费万5
cerebro.broker.setcommission(0.0005)

# 以发出信号当日收盘价成交
cerebro.broker.set_coc(True)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.AllInSizerInt, percents=99)

print('Starting Portfolio Value: {:.2f}'.format(cerebro.broker.getvalue()))

cerebro.addanalyzer(bt.analyzers.SQN)

result = cerebro.run()

print('Ending Portfolio Value: {:.2f}'.format(cerebro.broker.getvalue()))

In [ ]:
ana = result[0].analyzers.sqn.get_analysis()
print("sqn: {:.3f}, trades:{:d}".format(ana['sqn'],ana['trades']))

In [ ]:
cerebro.plot(iplot=True)

In [ ]:
from backtrader_plotting import Bokeh
from bokeh.plotting import output_file

output_file('docker_test.html')

plotconfig = {
    'r:^Broker.*': dict(
        plotlinelabels=True,
    ),
    'r:^Trades.*': dict(
        plotlinelabels=True,
    ),
    'r:MACDEXT': dict(
        plotlinelabels=True,
    ),
    'r:CDLXSIDEGAP3METHODS': dict(
        plot=False,
        subplot=True,
    ),
}

b = Bokeh(style='bar', plotconfig=plotconfig,output_mode='save')
cerebro.plot(b)